In [ ]:
# 📊 Data Cleaning Script
# This script loads, cleans, and merges multiple datasets related to math learning outcomes
# and contextual indicators during the COVID-19 pandemic.

In [ ]:
import pandas as pd
import pycountry
import os

In [ ]:
# -----------------------------
# Step 1: Load and clean Math Proficiency data
# -----------------------------
df_math = pd.read_csv("datasets/raw_data/minimum_proficiency.raw.csv")
df_math = df_math[["geoUnit", "year", "value"]]
df_math = df_math.rename(
    columns={"geoUnit": "ISO", "year": "Year", "value": "math_proficiency"}
)
df_math = df_math[df_math["Year"].isin([2019, 2023])]

In [ ]:
# Filter only valid countries by ISO code
valid_iso = set([c.alpha_3 for c in pycountry.countries])
df_math = df_math[df_math["ISO"].isin(valid_iso)].copy()

In [ ]:
# -----------------------------
# Step 2: Load and clean Completion Rate
# -----------------------------
df_completion = pd.read_csv("datasets/raw_data/completion_rate.raw.csv")
df_completion = df_completion[["geoUnit", "year", "value"]]
df_completion = df_completion.rename(
    columns={"geoUnit": "ISO", "year": "Year", "value": "completion_rate"}
)
df_completion = df_completion[df_completion["Year"].isin([2019, 2020, 2023])]

In [ ]:
# -----------------------------
# Step 3: Load and clean Government Spending
# -----------------------------
df_spending = pd.read_csv("datasets/raw_data/gov_expending.raw.csv")
df_spending = df_spending[["geoUnit", "year", "value"]]
df_spending = df_spending.rename(
    columns={"geoUnit": "ISO", "year": "Year", "value": "gov_edu_spending"}
)
df_spending = df_spending[df_spending["Year"].isin([2019, 2020, 2023])]

In [ ]:
# -----------------------------
# Step 4: Load and clean Out-of-School Rate
# -----------------------------
df_out_school = pd.read_csv("datasets/raw_data/out_of_school.raw.csv")
df_out_school = df_out_school[["geoUnit", "year", "value"]]
df_out_school = df_out_school.rename(
    columns={"geoUnit": "ISO", "year": "Year", "value": "out_of_school_rate"}
)
df_out_school = df_out_school[df_out_school["Year"].isin([2019, 2020, 2023])]

In [ ]:
# -----------------------------
# Step 5: Load and clean Pupil-to-Trained Teacher Ratio
# -----------------------------
df_teacher = pd.read_csv("datasets/raw_data/pupil_teacher_ratio.raw.csv")
df_teacher = df_teacher[["geoUnit", "year", "value"]]
df_teacher = df_teacher.rename(
    columns={"geoUnit": "ISO", "year": "Year", "value": "trained_teacher_ratio"}
)
df_teacher = df_teacher[df_teacher["Year"].isin([2019, 2020, 2023])]
df_teacher = df_teacher[df_teacher["ISO"].isin(valid_iso)].copy()

In [ ]:
# -----------------------------
# Step 6: Load and clean Digital Connectivity data
# -----------------------------
df_digital = pd.read_excel(
    "datasets/raw_data/School-Age-Digital-Connectivity.raw.xlsx", sheet_name="Primary"
)
df_digital = df_digital[df_digital["ISO3"].notna()]
df_digital = df_digital[["ISO3", "Total"]]
df_digital = df_digital.rename(columns={"ISO3": "ISO", "Total": "digital_connectivity"})

In [ ]:
# -----------------------------
# Step 7: Load and clean School Closure Duration data
# -----------------------------
df_closure = pd.read_excel(
    "datasets/raw_data/duration-of-school-closures.raw.xlsx", sheet_name="database"
)
df_closure = df_closure[["Country", "Days fully closed", "Days partially closed"]]
df_closure["school_closure_days"] = df_closure[
    ["Days fully closed", "Days partially closed"]
].sum(axis=1)

In [ ]:
# Fix country names to match pycountry
manual_fixes = {
    "Bolivia (Plurinational State of)": "Bolivia",
    "Iran (Islamic Republic of)": "Iran",
    "Republic of Korea": "South Korea",
    "Lao PDR": "Laos",
    "Palestine": "Palestine, State of",
    "Viet Nam": "Vietnam",
    "Russian Federation": "Russia",
    "Syrian Arab Republic": "Syria",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "United States of America": "United States",
    "Czechia": "Czech Republic",
    "Türkiye": "Turkey",
}
df_closure["Country_fixed"] = df_closure["Country"].str.strip().replace(manual_fixes)

In [ ]:
# Map country names to ISO codes
def get_iso(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

In [ ]:
df_closure["ISO"] = df_closure["Country_fixed"].apply(get_iso)
df_closure_cleaned = df_closure[["ISO", "school_closure_days"]].dropna()

In [ ]:
# -----------------------------
# Step 8: Load World Bank Income Classification
# -----------------------------
df_income = pd.read_csv("datasets/raw_data/worldbank_classification.csv")
df_income = df_income[["Code", "Income group"]]
df_income = df_income.rename(columns={"Code": "ISO", "Income group": "income_group"})

In [ ]:
# -----------------------------
# Step 9: Merge all datasets
# -----------------------------
df_final = df_math.copy()
df_final = df_final.merge(df_completion, on=["ISO", "Year"], how="left")
df_final = df_final.merge(df_spending, on=["ISO", "Year"], how="left")
df_final = df_final.merge(df_out_school, on=["ISO", "Year"], how="left")
df_final = df_final.merge(df_teacher, on=["ISO", "Year"], how="left")
df_final = df_final.merge(df_digital, on="ISO", how="left")
df_final = df_final.merge(df_closure_cleaned, on="ISO", how="left")
df_final = df_final.merge(df_income, on="ISO", how="left")

In [ ]:
# Keep only countries with valid income group classification
df_final_cleaned = df_final[df_final["income_group"].notna()].copy()

In [ ]:
# Round numerical columns to 2 decimal places
df_final_cleaned = df_final_cleaned.round(2)

In [ ]:
# -----------------------------
# Step 10: Save cleaned datasets
# -----------------------------
os.makedirs("datasets/cleaned_data", exist_ok=True)
df_math.to_csv("datasets/cleaned_data/math_proficiency.cleaned.csv", index=False)
df_completion.to_csv("datasets/cleaned_data/completion_rate.cleaned.csv", index=False)
df_spending.to_csv("datasets/cleaned_data/gov_spending.cleaned.csv", index=False)
df_out_school.to_csv("datasets/cleaned_data/out_of_school.cleaned.csv", index=False)
df_teacher.to_csv(
    "datasets/cleaned_data/trained_teacher_ratio.cleaned.csv", index=False
)
df_digital.to_csv("datasets/cleaned_data/digital_connectivity.cleaned.csv", index=False)
df_closure_cleaned.to_csv(
    "datasets/cleaned_data/school_closure.cleaned.csv", index=False
)
df_final_cleaned.to_csv("datasets/final_dataset.csv", index=False)